In [1]:
# Requerimientos
!pip install nltk spacy
!pip install nltk markovify
!pip install kagglehub


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
# Descargar modelos y recursos
import re
import math
import nltk
import markovify
import random
from nltk import ngrams, bigrams, FreqDist, word_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
from collections import defaultdict, Counter


import spacy
try:
    nlp_en = spacy.load("en_core_web_sm")
except:
    from spacy.cli import download
    download("en_core_web_sm")
    nlp_en = spacy.load("en_core_web_sm")

try:
    nlp_es = spacy.load("es_core_news_sm")
except:
    from spacy.cli import download
    download("es_core_news_sm")
    nlp_es = spacy.load("es_core_news_sm")

print("Recursos cargados correctamente.")

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 12.4 MB/s eta 0:00:0000:01



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 54.4 MB/s eta 0:00:0000:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


Recursos cargados correctamente.


In [3]:
# Corpus en español
corpus_es = """
El 16 de marzo de 2001, el fundador de Wikipedia, Jimmy Wales, anunció, a través de un mensaje de correo, su intención de internacionalizar el proyecto mediante la creación de ediciones en francés, alemán, español y, posiblemente, japonés. En el mismo mensaje comenta, bromeando, que también ha recibido una petición para crear una wiki en catalán.1​

Casi dos meses después, el 11 de mayo, los programadores Jason Richey y Toan Vo abrieron once wikis para iniciar las diferentes ediciones de wikipedias en varios idiomas, entre ellos el español.2​ La Wikipedia en esta lengua comenzó a funcionar el 20 de mayo de 2001. Entre sus primeros artículos aparecen «Países del mundo», creado el 21 de mayo de 2001, a las 00:19 UTC,3​ «Ayuda:Cómo empezar una página», creado un día más tarde,4​ e «Informática», el 25 de mayo.5​ A finales de ese año, el número de artículos ascendía a la cifra de 220,6​ pudiéndose leer entre ellos «Física de partículas», «Don Quijote de la Mancha», «Materia» y «Wiki».

Meses después, en febrero de 2002, la mayoría de los participantes de esta edición mostraron su desacuerdo con la propuesta, rechazada más tarde, de financiar Wikipedia mediante publicidad. En un principio, los dos fundadores, Jimmy Wales y Larry Sanger, se mostraron a favor de incluir publicidad.7​ Sanger estaba totalmente convencido de incluir anuncios y Wales rechazó las críticas de la comunidad hispanohablante sobre la inclusión de publicidad al escribir que «quiero hacer constar que pienso que dicha gente está seriamente equivocada en muchos aspectos de su visión del mundo. La inclusión de publicidad debería hacerse con buen gusto».8​ Muchos de los editores españoles se separaron del proyecto para crear una bifurcación del proyecto, bautizada como Enciclopedia Libre Universal en Español, la cual, desde su implantación en febrero de 2002 y hasta mediados de 2004, llegó a tener una mayor cantidad de artículos que Wikipedia en español. Las razones esgrimidas para ello fueron «su rechazo a la censura, a la existencia de una línea editorial y a la inclusión de publicidad» y al hecho de que por entonces la inclusión de publicidad dependía de la empresa propietaria,9​ Bomis Inc, una sociedad mercantil propiedad de Wales.


Primer artículo de la Wikipedia en español del que se ha hallado registro, «Países del mundo», creado por un usuario anónimo el 21 de mayo de 2001, a las 20:19. Gracias a que Wikipedia conserva un registro de las ediciones, es posible verificar tal edición.
Esta protesta de los wikipedistas españoles e hispanohablantes tuvo un papel relevante para que la controversia se zanjara a favor de la exclusión definitiva de la publicidad como forma de financiar el proyecto, ya fuera por el temor de Wales al efecto contagio en otros idiomas o por razones éticas.10​11​ Para alguno de los editores implicados en la bifurcación, esta podría haber sido «la gota que colmó la balanza a favor de una Wikipedia 100% libre de publicidad», mientras que Wales declararía posteriormente que «la bifurcación española fue un hecho importante en la historia de Wikipedia, pero no provocó un cambio».7​ Todas las subsiguientes propuestas sobre la financiación de Wikipedia por medio de publicidad han sido rápida y firmemente rechazadas, también por el propio fundador Jimmy Wales. Este último, sin embargo, en ocasiones ha declarado a la prensa que no descarta por completo la idea.12​

Durante un tiempo tras su escisión, la versión en castellano de Wikipedia tuvo una actividad muy escasa. En octubre de 2002 y tras producirse la actualización a la Fase III del software, que pasaría a llamarse MediaWiki, el número de usuarios comenzó de nuevo a incrementarse. En marzo de 2006, Wikipedia en español era ya el más activo de los dos proyectos. Desde entonces, muchos usuarios colaboran en ambos y buena parte de los artículos se traducen entre versiones de Wikipedia, dada la compatibilidad de las licencias.

En noviembre de 2003 se tomó la decisión de continuar con el nombre oficial para la versión en castellano, «Wikipedia», en una votación en la que se barajaron nombres tales como «Librepedia», «Huiquipedia» o «Ñiquipedia».13​ Esta votación presentó la novedad de contar con la primera edición (sobre un total de 13 hasta mayo de 2008) de Jimbo Wales (disc. · contr.) en Wikipedia en español. Jimbo Wales votó a favor del nombre «Wikipedia».13​

Clasificación por número de artículos
Fue durante mucho tiempo, la octava edición más grande en cuanto al número de artículos, hasta que Wikipedia en portugués la sobrepasó en mayo de 2005 y, posteriormente, Wikipedia en italiano en agosto del mismo año, quedando en la décima posición. La tendencia ascendente no llegaría hasta abril de 2007, fecha en la que Wikipedia en español comenzó a escalar posiciones, superando poco a poco a otras versiones. Ese mes logró alcanzar a Wikipedia en sueco, pasando a ocupar de nuevo el noveno lugar de la lista.

El 5 de julio de 2009, la versión en castellano de Wikipedia volvió a superar a su homóloga en portugués, recuperando así el octavo lugar en número de artículos, pero solo se mantuvo por dos días. El 7 de julio, Wikipedia en portugués se adelantó a la versión en español, aunque tan solo por unas horas. Un año más tarde, el 7 de junio de 2010, Wikipedia en español superó a Wikipedia en neerlandés y se convirtió en la séptima en número de artículos. El 22 de marzo de 2011, Wikipedia en español sobrepasó a Wikipedia en japonés, pasando a ocupar el sexto lugar. Desde el 26 de octubre de 2011, tras haber superado a Wikipedia en polaco, y hasta el 30 de octubre de 2011, fue la quinta en cuanto al número de artículos. En esa fecha fue superada por la versión neerlandesa, que registraba un aumento rápido, casi exclusivamente por el uso de bots que automatizaban la creación de centenares de miles de artículos pequeños. El 21 de noviembre del mismo año volvió a ser superada, por un día, por Wikipedia en polaco, de la que se mantenía a poca distancia tras haberla superado el mes anterior.

Tras superar a su homóloga polaca, se consolidó por poco tiempo como la sexta Wikipedia en cantidad de artículos. Finalmente, Wikipedia en polaco superó a Wikipedia en español, que pasó nuevamente al séptimo puesto. Un año más tarde, el 9 de octubre de 2012, Wikipedia en español volvió por breve espacio de tiempo a sobrepasar a Wikipedia en polaco y nuevamente se ubicó en la sexta posición; esa posición fue disputada por Wikipedia polaca a lo largo del mismo día y los siguientes, superando a Wikipedia en español por unos pocos artículos, manteniéndose ambas Wikipedias a corta distancia, hasta el día 15 del mismo mes, en la que de nuevo Wikipedia en español supera a la polaca. Permaneció en la sexta posición durante dos meses y medio, antes de regresar de nuevo brevemente al séptimo puesto el 25 de diciembre de ese mismo año tras haber sido superada por la Wikipedia en ruso, que había mostrado un rápido crecimiento sostenido en los meses anteriores.

La situación fue restaurada el 6 de enero de 2013. El 27 de febrero, la versión en español fue de nuevo superada por la versión en ruso, volviendo a ocupar la séptima plaza. El 16 de mayo de 2013, Wikipedia en español alcanza un millón de artículos, supera a la Wikipedia en ruso y regresa a la sexta posición. Se convirtió así en la séptima edición de Wikipedia en incluir un millón de artículos, tras haberlo conseguido antes las ediciones de la Wikipedia en inglés, que el 1 de marzo de 2006 se convirtió en la primera en superar tal cifra de artículos, seguida el 27 de diciembre de 2009 por la Wikipedia en alemán, el 21 de septiembre de 2010 por la Wikipedia en francés, el 17 de diciembre de 2011 por la Wikipedia en neerlandés, el 22 de enero de 2013 por la Wikipedia en italiano y el 11 de mayo del mismo año por la Wikipedia en ruso. Tras permanecer en esa posición un mes, la Wikipedia en sueco superó a la versión en español el 19 de junio de 2013, dejándola en la séptima posición. Conservó esa plaza durante varios meses, hasta que el 2 de octubre la Wikipedia en ruso volvió a ganar en número de artículos a la edición en español, que pasó a la octava posición (no ocupada desde 2009). A finales de agosto de 2014, la Wikipedia en cebuano hizo pasar a la edición en castellano a la novena plaza. Con posterioridad, en septiembre del mismo año, el crecimiento de la Wikipedia en samareño hizo pasar a la edición en español a la décima plaza, posición que ya había llegado a ocupar en el período entre 2005 y 2007.
"""

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jmorenobl/corpus-de-la-wikipedia-en-espaol")

print("Path to dataset files:", path)

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 1.36G/1.36G [00:24<00:00, 59.1MB/s]

Extracting files...


Path to dataset files: /home/codespace/.cache/kagglehub/datasets/jmorenobl/corpus-de-la-wikipedia-en-espaol/versions/2


In [ ]:
# with open(path + "/eswiki-latest-pages-articles.txt") as f:
#     corpus_es = f.read()

# corpus_es = []
# with open(path + "/eswiki-latest-pages-articles.txt", "r", encoding="utf-8") as f:
#     while chunk := f.read(10**6):  # Leer en bloques de 1MB
#         corpus_es.append(chunk)

# corpus_es = "".join(corpus_es)

# def read_large_file(file_path, chunk_size=10**6):
#     with open(file_path, "r", encoding="utf-8") as f:
#         while chunk := f.read(chunk_size):  
#             yield chunk  # Devuelve cada fragmento sin acumular en memoria

# file_path = path + "/eswiki-latest-pages-articles.txt"
# corpus_es = (chunk for chunk in read_large_file(file_path))


import os

def read_percent_file(file_path, percent=1):
    file_size = os.path.getsize(file_path)  # Tamaño total en bytes
    max_bytes = file_size // 10  # 10% del tamaño total

    read_bytes = 0
    with open(file_path, "r", encoding="utf-8") as f:
        while read_bytes < max_bytes and (chunk := f.read(10**6)):
            yield chunk
            read_bytes += len(chunk)

file_path = path + "/eswiki-latest-pages-articles.txt"
corpus_es = (chunk for chunk in read_percent_file(file_path, 1))

: 

In [ ]:
# Crear un solo modelo con state_size=2
modelo_es = markovify.Text(corpus_es, state_size=2)

In [ ]:
# Generar 5 oraciones con parámetros para más variedad
print("=== Oraciones generadas en español ===")
for _ in range(5):
    oracion_es = modelo_es.make_sentence(tries=1000, max_overlap_ratio=0.4, max_words=100)
    if oracion_es:
        print(f"Texto generado: {oracion_es}")
    else:
        print("No se pudo generar una oración válida (intenta con un corpus más grande).")

=== Oraciones generadas en español ===
Texto generado: Su desarrollo tuvo lugar a un individuo o de otros lugares.
Texto generado: En 1990 se habilitó una de ellas doradas, que enmarcaron primitivamente el patio de gallos.
Texto generado: No disponían de un estadio de Huracán en la derrota, Casanova mantuvo los límites entre las ciudades de Irlanda en uno de los superhéroes más populares fueron Austria, España, Francia, Alemania, Polonia, Portugal, Reino Unido, Thomas Hobbes, un amigo con Talita, pero siente que el siglo V a.
Texto generado: Homero ilustró tanto a la marinería y comercio electrónico.
Texto generado: Los productos provienen de patrocinadores y de los godos lanzaron un ataque sobre los elementos autóctonos del México provincial, hasta que su alma que puede ser un eremita ascético en una ciudad persistente en la ley de Hooke, la Royal Society «por su clima generalmente templado, aunque los analizadores léxicos se agrupan en una computadora, puede integrar y asimilar nueva